In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
EVALUATE_ONLY = True
TEST_ON_LARGE_DATASET = True
TISSUE_CLASSES = ('ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM')
DATASETS_LINKS = {
    'train': '1XtQzVQ5XbrfxpLHJuL0XBGJ5U7CS-cLi',
    'train_small': '1qd45xXfDwdZjktLFwQb-et-mAaFeCzOR',
    'train_tiny': '1I-2ZOuXLd4QwhZQQltp817Kn3J0Xgbui',
    'test': '1RfPou3pFKpuHDJZ-D9XDFzgvwpUBFlDr',
    'test_small': '1wbRsog0n7uGlHIPGLhyN-PMeT2kdQ2lI',
    'test_tiny': '1viiB0s041CNsAK4itvX8PnYthJ-MDnQc'
}

In [ ]:
from pathlib import Path
import numpy as np
from typing import List
from tqdm.notebook import tqdm
from time import sleep
from PIL import Image
import IPython.display
from sklearn.metrics import balanced_accuracy_score
import gdown

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

train_data_path = '/content/drive/MyDrive/train.npz'
test_data_path = '/content/drive/MyDrive/test.npz'

train_data = np.load(train_data_path)
test_data = np.load(test_data_path)

X_train, y_train = train_data['data'], train_data['labels']
X_test, y_test = test_data['data'], test_data['labels']

print("Train data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

input_shape = (64, 64, X_train.shape[-1])
X_train_resized = np.array([tf.image.resize(img, input_shape[:2]).numpy() for img in X_train])
X_test_resized = np.array([tf.image.resize(img, input_shape[:2]).numpy() for img in X_test])

X_train_resized = X_train_resized / 255.0
X_test_resized = X_test_resized / 255.0

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train_resized)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

from sklearn.model_selection import train_test_split
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_resized, y_train, test_size=0.2, random_state=42)

history = model.fit(datagen.flow(X_train_final, y_train_final, batch_size=32),
                    epochs=50,
                    validation_data=datagen.flow(X_val, y_val, batch_size=64),
                    callbacks=[early_stopping, lr_scheduler])
#оценка
test_loss, test_accuracy = model.evaluate(X_test_resized, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

model.save('CNN_or_somth.h5')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train data shape: (18000, 224, 224, 3) (18000,)
Test data shape: (4500, 224, 224, 3) (4500,)
Epoch 1/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 209s 456ms/step - accuracy: 0.4256 - loss: 2.0043 - val_accuracy: 0.2197 - val_loss: 8.9262 - learning_rate: 0.0010
Epoch 2/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 257s 446ms/step - accuracy: 0.5901 - loss: 1.1117 - val_accuracy: 0.1986 - val_loss: 23.2822 - learning_rate: 0.0010
Epoch 3/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 190s 421ms/step - accuracy: 0.6180 - loss: 1.0148 - val_accuracy: 0.3961 - val_loss: 3.4084 - learning_rate: 0.0010
Epoch 4/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 192s 425ms/step - accuracy: 0.6695 - loss: 0.8943 - val_accuracy: 0.2575 - val_loss: 10.9275 - learning_rate: 0.0010
Epoch 5/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 199s 442ms/step - accuracy: 0.6966 - loss: 0.8235 - val_accuracy: 0.5067 - val_loss: 2.0123 - learning_rate: 0.0

Test Accuracy: 53.80%


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

train_data_path = '/content/drive/MyDrive/train.npz'
test_data_path = '/content/drive/MyDrive/test.npz'

train_data = np.load(train_data_path)
test_data = np.load(test_data_path)

X_train, y_train = train_data['data'], train_data['labels']
X_test, y_test = test_data['data'], test_data['labels']

print("Train data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

input_shape = (64, 64, X_train.shape[-1])
X_train_resized = np.array([tf.image.resize(img, input_shape[:2]).numpy() for img in X_train])
X_test_resized = np.array([tf.image.resize(img, input_shape[:2]).numpy() for img in X_test])

X_train_resized = X_train_resized / 255.0
X_test_resized = X_test_resized / 255.0

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train_resized)

#Модель CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
#обучение
from sklearn.model_selection import train_test_split
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_resized, y_train, test_size=0.2, random_state=42)

history = model.fit(datagen.flow(X_train_final, y_train_final, batch_size=32),
                    epochs=30,
                    validation_data=datagen.flow(X_val, y_val, batch_size=32),
                    callbacks=[early_stopping, lr_scheduler])

test_loss, test_accuracy = model.evaluate(X_test_resized, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

model.save('/content/drive/MyDrive/CNN_or_s.h5')

def load_model(model_path):
    return tf.keras.models.load_model(model_path)

loaded_model = load_model('/content/drive/MyDrive/CNN_or_s.h5')

#оценка
loaded_test_loss, loaded_test_accuracy = loaded_model.evaluate(X_test_resized, y_test)
print(f"Loaded Model Test Accuracy: {loaded_test_accuracy * 100:.2f}%")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train data shape: (18000, 224, 224, 3) (18000,)
Test data shape: (4500, 224, 224, 3) (4500,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


450/450 ━━━━━━━━━━━━━━━━━━━━ 215s 455ms/step - accuracy: 0.1961 - loss: 3.9852 - val_accuracy: 0.1689 - val_loss: 3.2401 - learning_rate: 1.0000e-05
Epoch 2/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 201s 445ms/step - accuracy: 0.3153 - loss: 2.3238 - val_accuracy: 0.4642 - val_loss: 1.5159 - learning_rate: 1.0000e-05
Epoch 3/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 200s 442ms/step - accuracy: 0.3742 - loss: 1.8292 - val_accuracy: 0.4853 - val_loss: 1.4744 - learning_rate: 1.0000e-05
Epoch 4/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 193s 421ms/step - accuracy: 0.3988 - loss: 1.6906 - val_accuracy: 0.5083 - val_loss: 1.4335 - learning_rate: 1.0000e-05
Epoch 5/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 193s 428ms/step - accuracy: 0.4122 - loss: 1.5894 - val_accuracy: 0.5017 - val_loss: 1.4484 - learning_rate: 1.0000e-05
Epoch 6/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 190s 422ms/step - accuracy: 0.4410 - loss: 1.4987 - val_accuracy: 0.5119 - val_loss: 1.4487 - learning_rate: 1.0000e-05
141/141 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accura

Test Accuracy: 49.64%


141/141 ━━━━━━━━━━━━━━━━━━━━ 18s 112ms/step - accuracy: 0.6859 - loss: 0.9139
Loaded Model Test Accuracy: 49.64%
